# Aprendizaje por refuerzo (posponer al viernes)

Instalar las siguientes librerias para usar esta notebook:

!pip install cmake pygame gym[all] pip stable_baselines3

Referencia: https://www.gocoder.one/blog/rl-tutorial-with-openai-gym

![](https://raw.githubusercontent.com/igomezv/DataScienceIntermedio/main/img/refuerzo.png)
Fuente de la imagen: medium.com

![](https://raw.githubusercontent.com/igomezv/DataScienceIntermedio/main/img/RLdog.jpg)
Fuente: kdnuggets.com

Objetivo: maximizar la recompenza.

**Definiciones importantes:**

- Agente: modelo a entrenar para que tome decisiones.
- Ambiente: Entorno donde interactúa el agente. El ambiente limita y pone reglas. 

Entre ambiente y agente hay las siguientes relaciones: 

- Acción: posibles acciones que puede tomar el agente en determinado momento.
- Estado (del ambiente): indicadores del ambiente sobre cómo están los diversos elementos que lo componen en determinado momento.
- Recompensas (o penalización): cada acción del agente obtiene un premio o una penalización. 

Ver: https://towardsdatascience.com/hands-on-introduction-to-reinforcement-learning-in-python-da07f7aaca88 


![](https://raw.githubusercontent.com/igomezv/DataScienceIntermedio/main/img/RLalg.png)
Fuente: medium.com

#### Usar el entorno TAXI

**Contexto:** Taxi es un entorno disponible en Open-IA gym (https://www.gymlibrary.dev/environments/toy_text/taxi/).

- El objetivo de Taxi es recoger pasajeros y dejarlos en el destino en la menor cantidad de movimientos. 

- Se comparara un agente de taxis que realiza acciones al azar con uno entrenado mediante Aprendizaje Reforzado.

- Acciones: moverse hacia arriba, abajo, izquierda, derecha, recoger o dejar pasajeros.

Descripción de la documentación:

Hay cuatro ubicaciones designadas en el mundo de la cuadrícula indicadas por R (rojo), G (verde), Y (amarillo) y B (azul). Cuando el episodio inicia, el taxi parte de una casilla aleatoria y el pasajero se encuentra en una ubicación aleatoria. El taxi conduce hasta la ubicación del pasajero, lo recoge, conduce hasta el destino del pasajero (otra de las cuatro ubicaciones especificadas) y luego deja al pasajero. Una vez que se deja al pasajero, el episodio termina.

Map:

+---------+

|R: | : :G|

| : | : : |

| : : : : |

| | : | : |

|Y| : |B: |

+---------+

##### Comportamiento


Hay 6 acciones deterministas discretas:

    0: mover al sur

    1: mover al norte

    2: moverse hacia el este

    3: muévete hacia el oeste

    4: pasajero de recogida

    5: dejar al pasajero

Observaciones

Hay 500 estados discretos ya que hay 25 posiciones de taxi, 5 ubicaciones posibles del pasajero (incluido el caso cuando el pasajero está en el taxi) y 4 ubicaciones de destino.

Tenga en cuenta que hay 400 estados a los que se puede llegar durante un episodio. Los estados faltantes corresponden a situaciones en las que el pasajero se encuentra en el mismo lugar que su destino, ya que esto suele indicar el final de un episodio. Se pueden observar cuatro estados adicionales justo después de un episodio exitoso, cuando tanto el pasajero como el taxi están en el destino. Esto da un total de 404 estados discretos alcanzables.

Cada espacio de estado está representado por la tupla: (fila_taxi, col_taxi, ubicación_pasajero, destino)


Ubicaciones de pasajeros:

    0: R (rojo)

    1: G (verde)

    2: Y (amarillo)

    3: B (azul)

    4: en taxi

Destinos:

    0: R (rojo)

    1: G (verde)

    2: Y (amarillo)

    3: B (azul)

Recompensas

     -1 por paso a menos que se active otra recompensa.

     +20 entregando pasajero.

     -10 ejecutar acciones de “recogida” y “devolución” de forma ilegal.
     
     - Una ilegalidad consiste en ejecutar las acciones"recoger"o "devolver" en lugares donde no hay personas.

![](https://www.gocoder.one/static/state-rewards-62ab43a53e07062b531b3199a8bab5b3.png)

Como llegar al punto R sin perder tantos puntos?

## Q-Learning

### Tablas-Q

Una tabla Q es una tabla de búsqueda que almacena valores que representan las recompensas futuras máximas esperadas que el agente puede esperar por una acción determinada en un estado determinado (conocidos como valores Q).

![](https://i.stack.imgur.com/Bn6MY.gif)

Estas tablas se suelen inicializar en 0s

qtable = np.zeros((state_size, action_size))


Hyperparámetros:

- Tasa de aprendizaje (α): qué tanto el agente debe aceptar nueva información sobre la información previamente aprendida

- Factor de descuento (γ): qué tanto el agente debe considerar las recompensas que podría recibir en el futuro frente a su recompensa inmediata.

#### Primero, analicemos el comportamiento aleatorio

In [1]:
import gym

In [2]:
env = gym.make('Taxi-v3')

In [3]:
# create a new instance of taxi, and get the initial state
state = env.reset()

In [4]:
import gym
import numpy as np
import random

def randomtaxi():
    total_reward = 0 
    # crear entorno de Taxi
    env = gym.make('Taxi-v3')

    # crear una instancia de Taxi, y obtener un estado inicial
    state = env.reset()

    num_steps = 99

    for s in range(num_steps+1):
        print("Paso: {}/{}".format(s+1, num_steps+1))

        # muestrear una accion aleatoria de la lista de acciones posibles
        action = env.action_space.sample()
        # ejecutar esta accion sobre el entorno
        new_state, reward, done, info = env.step(action)
        # Update to our new state
        state = new_state
        total_reward += reward 
        print("Accion {} Estado {} Recompensa {} Total rec {}".format(action, state, reward, total_reward))


        # imprimir nuevo estado
        env.render()  

        # if done, finish episode
        if done == True:
            break        
            
            
    # terminar y cerrar el entorno del taxi
    env.close()
    return total_reward 

In [5]:
randomtaxi()

Paso: 1/100
Accion 1 Estado 369 Recompensa -1 Total rec -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Paso: 2/100
Accion 3 Estado 369 Recompensa -1 Total rec -2
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Paso: 3/100
Accion 2 Estado 389 Recompensa -1 Total rec -3
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Paso: 4/100
Accion 4 Estado 389 Recompensa -10 Total rec -13
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
Paso: 5/100
Accion 3 Estado 369 Recompensa -1 Total rec -14
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
Paso: 6/100
Accion 2 Estado 389 Recompensa -1 Total rec -15
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Paso: 7/100
Accion 2 Estado 389 Recompensa -1 Total rec -16
+---------+
|R: | : :

-406

In [6]:
import numpy as np
import gym
import random

def rltaxi():

    # create Taxi environment
    env = gym.make('Taxi-v3')

    # initialize q-table
    state_size = env.observation_space.n
    action_size = env.action_space.n
    qtable = np.zeros((state_size, action_size))

    # hyperparameters
    learning_rate = 0.9
    discount_rate = 0.8
    epsilon = 1.0
    decay_rate= 0.005

    # training variables
    num_episodes = 1000
    max_steps = 99 # per episode

    # training
    for episode in range(num_episodes):

        # reset the environment
        state = env.reset()
        done = False

        for s in range(max_steps):

            # exploration-exploitation tradeoff
            if random.uniform(0,1) < epsilon:
                # explore
                action = env.action_space.sample()
            else:
                # exploit
                action = np.argmax(qtable[state,:])

            # take action and observe reward
            new_state, reward, done, info = env.step(action)

            # Q-learning algorithm
            qtable[state,action] = qtable[state,action] + learning_rate * (reward + discount_rate * np.max(qtable[new_state,:])-qtable[state,action])

            # Update to our new state
            state = new_state

            # if done, finish episode
            if done == True:
                break

        # Decrease epsilon
        epsilon = np.exp(-decay_rate*episode)

    print(f"Training completed over {num_episodes} episodes")
    input("Press Enter to watch trained agent...")

    # watch trained agent
    state = env.reset()
    done = False
    total_reward = 0

    for s in range(max_steps+1):

        print(f"TRAINED AGENT")
        print("Paso {}".format(s+1))

        action = np.argmax(qtable[state,:])
        new_state, reward, done, info = env.step(action)
        total_reward += reward
        env.render()
        total_reward += reward 
        print("Accion {} Estado {} Recompensa {} Total rec {}".format(action, state, reward, total_reward))
        state = new_state
#comentar esto y volver a correr
       if done == True:
           break

    env.close()


In [7]:
rltaxi()

Training completed over 1000 episodes
Press Enter to watch trained agent...
TRAINED AGENT
Paso 1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Accion 0 Estado 33 Recompensa -1 Total rec -2
TRAINED AGENT
Paso 2
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Accion 0 Estado 133 Recompensa -1 Total rec -4
TRAINED AGENT
Paso 3
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Accion 2 Estado 233 Recompensa -1 Total rec -6
TRAINED AGENT
Paso 4
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Accion 2 Estado 253 Recompensa -1 Total rec -8
TRAINED AGENT
Paso 5
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Accion 0 Estado 273 Recompensa -1 Total rec -10
TRAINED AGENT
Paso 6
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
A

###### Ejercicio comparar recompensas de ambos metodos con 3 diferentes numeros de iteraciones.